In [ ]:
%pip install --quiet --upgrade langchain-openai azure-identity

In [1]:
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from azure.identity import DefaultAzureCredential
from langchain_openai import AzureChatOpenAI

import os


# Get the Azure Credential make sure to assign yourself the role Cognitive Services OpenAI User
credential = DefaultAzureCredential()

# Set the API type to `azure_ad`
os.environ["OPENAI_API_TYPE"] = "azure_ad"
# Set the API_KEY to the token from the Azure credential
os.environ["OPENAI_API_KEY"] = credential.get_token("https://cognitiveservices.azure.com/.default").token
os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://aoaise.openai.azure.com/"

llm = AzureChatOpenAI(
    deployment_name="gpt35turbo16"
)

In [2]:
# Run the LLM
llm.invoke("Tell me a joke")

AIMessage(content="Sure, here's a classic one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 11, 'total_tokens': 32}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-83df2cb9-387b-47ca-a00a-8105c32f7435-0')

In [3]:
tool = FileManagementToolkit(
    root_dir=str("/home/wuerzelchen/Dev/Workshops/Ai"),
    selected_tools=["read_file", "write_file", "list_directory"],
)  # If you don't provide a root_dir, operations will default to the current working directory
tools = tool.get_tools()


In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the filemanagement tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [5]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what files are in my folder?"})



> Entering new AgentExecutor chain...

Invoking: `list_directory` with `{}`


example.txt
04_acs-lc-python.ipynb
02_agent_example.ipynb
01_tool_example.ipynb
requirements.txt
movies.csv
03_custom_tool_example.ipynb
.envThe files in your folder are:

1. example.txt
2. 04_acs-lc-python.ipynb
3. 02_agent_example.ipynb
4. 01_tool_example.ipynb
5. requirements.txt
6. movies.csv
7. 03_custom_tool_example.ipynb
8. .env

> Finished chain.


{'input': 'what files are in my folder?',
 'output': 'The files in your folder are:\n\n1. example.txt\n2. 04_acs-lc-python.ipynb\n3. 02_agent_example.ipynb\n4. 01_tool_example.ipynb\n5. requirements.txt\n6. movies.csv\n7. 03_custom_tool_example.ipynb\n8. .env'}

In [7]:
agent_executor.invoke({"input": "does README.md exist? it should include a title which welcomes the user and some basic information about tools and agents from langchain. Can you create it for me? If it already exists, please update it with the author if there is no info about the author, the author is wolflad@microsoft.com."})



> Entering new AgentExecutor chain...

Invoking: `list_directory` with `{'dir_path': '.'}`


README.md
example.txt
04_acs-lc-python.ipynb
02_agent_example.ipynb
01_tool_example.ipynb
requirements.txt
movies.csv
03_custom_tool_example.ipynb
.env
Invoking: `read_file` with `{'file_path': 'README.md'}`


# Welcome to Langchain

This README.md file includes some basic information about the tools and agents from Langchain.

## Tools

Langchain provides a set of tools that can be used to perform various tasks. These tools are designed to simplify the development and execution of blockchain applications.

## Agents

Langchain also includes a collection of agents that can be used to interact with blockchain networks. These agents provide a high-level interface for performing common operations such as querying the blockchain, creating transactions, and managing identities.

Invoking: `write_file` with `{'file_path': 'README.md', 'text': '# Welcome to Langchain\n\nThis README.md file includes 

{'input': 'does README.md exist? it should include a title which welcomes the user and some basic information about tools and agents from langchain. Can you create it for me? If it already exists, please update it with the author if there is no info about the author, the author is wolflad@microsoft.com.',
 'output': 'The README.md file has been created and updated with the author information. You can find it [here](sandbox:/README.md).'}